In [1]:
import random

import numpy as np
import torch
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from torch import nn
from tqdm import tqdm
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [2]:
data = np.load(r'D:\Advancing_ocean_intuition\pacific\last\1day_for_1day_all_variables_intuition_nino_area_surface.npz')
print(
    data.files)  # ['sst_obser', 'slfh', 'sshf', 'ssr', 'str', 'mld', 'analysis_temp', 'u', 'v', 'T_d', 'u_d', 'v_d', 'xx', 'yy', 'dT_dt', 'dT_dx', 'dT_dy']

sst_obser = data['sst_obser'][:, :40, :200]  # (9861, 41, 201)  #
print(sst_obser.shape)  # (9861, 41, 201)
slfh = data['slfh'][:, :40, :200]   # (9861, 41, 201)
sshf = data['sshf'][:, :40, :200]   # (9861, 41, 201)
ssr = data['ssr'][:, :40, :200]   # (9861, 41, 201)   # 可惜这个是 0
str = data['str'][:, :40, :200]   # (9861, 41, 201)
mld = data['mld'][:, :40, :200]   # (9861, 41, 201)
analysis_temp = data['analysis_temp'][:, :40, :200]  # (9861, 41, 201)
u = data['u'][:, :40, :200]  # (9861, 41, 201)
v = data['v'][:, :40, :200]  # (9861, 41, 201)
T_d = data['T_d'][:, :40, :200]  # (9861, 41, 201)
u_d = data['u_d'][:, :40, :200]  # (9861, 41, 201)
v_d = data['v_d'][:, :40, :200]  # (9861, 41, 201)
xx = data['xx'][:, :40, :200]  # (9861, 41, 201)
yy = data['yy'][:, :40, :200]  # (9861, 41, 201)
print(np.min(T_d))
print(np.max(T_d))
# print(xx)  # -5 -4.75 ... 4.75
# print(yy)  # 190  190.25 ... 239.75

dT_dt = data['dT_dt'][:, :40, :200]
dT_dx = data['dT_dx'][:, :40, :200]
dT_dy = data['dT_dy'][:, :40, :200]

['sst_obser', 'slfh', 'sshf', 'ssr', 'str', 'mld', 'analysis_temp', 'u', 'v', 'T_d', 'u_d', 'v_d', 'xx', 'yy', 'dT_dt', 'dT_dx', 'dT_dy']
(9861, 40, 200)
19.893036
31.218864


In [3]:
Q_net = (slfh + sshf + ssr + str)

In [4]:
data1 = np.load(r'D:\heat_wave\pacific\last\wind_93_19_pacific_area.npz')
print(data1.files) #['time', 'lat', 'lon', 'eastward_wind', 'northward_wind', 'wind_vector_divergence', 'wind_stress', 'surface_downward_eastward_stress', 'surface_downward_northward_stress', 'wind_stress_curl', 'wind_stress_divergence', 'wind_speed_rms', 'eastward_wind_rms', 'northward_wind_rms', 'sampling_length', 'surface_type']
eastward_wind = data1['eastward_wind'][:,:40,:200] # (9848, 66, 49)
northward_wind = data1['northward_wind'][:,:40,:200] # (9848, 66, 49)
surface_downward_eastward_stress = data1['surface_downward_eastward_stress'][:,:40,:200] # (9848, 66, 49)
surface_downward_northward_stress = data1['surface_downward_northward_stress'][:,:40,:200] # (9848, 66, 49)

['time', 'lat', 'lon', 'eastward_wind', 'northward_wind', 'wind_vector_divergence', 'wind_stress', 'surface_downward_eastward_stress', 'surface_downward_northward_stress', 'wind_stress_curl', 'wind_stress_divergence', 'wind_speed_rms', 'eastward_wind_rms', 'northward_wind_rms', 'sampling_length', 'surface_type']


In [5]:
# MAM   93
date_93_1 = 59
date_93_2 = 151
data_1 = date_93_1
data_2 = date_93_2

Q_net_93 = Q_net[data_1:data_2,:,:]
sst_obser_93 = sst_obser[data_1:data_2,:,:]
mld_93 = mld[data_1:data_2,:,:]
analysis_temp_93 = analysis_temp[data_1:data_2,:,:]
u_93 = u[data_1:data_2,:,:]
v_93 = v[data_1:data_2,:,:]
u_d_93 = u_d[data_1:data_2,:,:]
v_d_93 = v_d[data_1:data_2,:,:]
dT_dt_93 = dT_dt[data_1:data_2,:,:]
dT_dx_93 = dT_dx[data_1:data_2,:,:]
dT_dy_93 = dT_dy[data_1:data_2,:,:]
T_d_93 = T_d[data_1:data_2,:,:]
eastward_wind_93 = eastward_wind[data_1:data_2,:,:]
northward_wind_93 = northward_wind [data_1:data_2,:,:]
surface_downward_eastward_stress_93 = surface_downward_eastward_stress[data_1:data_2,:,:]
surface_downward_northward_stress_93 = surface_downward_northward_stress[data_1:data_2,:,:]

In [6]:
# MAM   94
date_94_1 = 424
date_94_2 = 516
data_1 = date_94_1
data_2 = date_94_2

u_94 = u[data_1:data_2,:,:]
v_94 = v[data_1:data_2,:,:]
v_d_94 = v_d[data_1:data_2,:,:]
T_d_94 = T_d[data_1:data_2,:,:]
u_d_94 = u_d[data_1:data_2,:,:]
mld_94 = mld[data_1:data_2,:,:]
dT_dt_94 = dT_dt[data_1:data_2,:,:]
dT_dx_94 = dT_dx[data_1:data_2,:,:]
dT_dy_94 = dT_dy[data_1:data_2,:,:]
Q_net_94 = Q_net[data_1:data_2,:,:]
sst_obser_94 = sst_obser[data_1:data_2,:,:]
analysis_temp_94 = analysis_temp[data_1:data_2,:,:]
eastward_wind_94 = eastward_wind[data_1:data_2,:,:]
northward_wind_94 = northward_wind [data_1:data_2,:,:]
surface_downward_eastward_stress_94 = surface_downward_eastward_stress[data_1:data_2,:,:]
surface_downward_northward_stress_94 = surface_downward_northward_stress[data_1:data_2,:,:]

In [7]:
# MAM   95
date_95_1 = 789
date_95_2 = 881
data_1 = date_95_1
data_2 = date_95_2

u_95 = u[data_1:data_2,:,:]
v_95 = v[data_1:data_2,:,:]
v_d_95 = v_d[data_1:data_2,:,:]
T_d_95 = T_d[data_1:data_2,:,:]
u_d_95 = u_d[data_1:data_2,:,:]
mld_95 = mld[data_1:data_2,:,:]
dT_dt_95 = dT_dt[data_1:data_2,:,:]
dT_dx_95 = dT_dx[data_1:data_2,:,:]
dT_dy_95 = dT_dy[data_1:data_2,:,:]
Q_net_95 = Q_net[data_1:data_2,:,:]
sst_obser_95 = sst_obser[data_1:data_2,:,:]
analysis_temp_95 = analysis_temp[data_1:data_2,:,:]
eastward_wind_95 = eastward_wind[data_1:data_2,:,:]
northward_wind_95 = northward_wind [data_1:data_2,:,:]
surface_downward_eastward_stress_95 = surface_downward_eastward_stress[data_1:data_2,:,:]
surface_downward_northward_stress_95 = surface_downward_northward_stress[data_1:data_2,:,:]

In [8]:
# MAM   96
date_96_1 = 1155
date_96_2 = 1247
data_1 = date_96_1
data_2 = date_96_2

u_96 = u[data_1:data_2,:,:]
v_96 = v[data_1:data_2,:,:]
v_d_96 = v_d[data_1:data_2,:,:]
T_d_96 = T_d[data_1:data_2,:,:]
u_d_96 = u_d[data_1:data_2,:,:]
mld_96 = mld[data_1:data_2,:,:]
dT_dt_96 = dT_dt[data_1:data_2,:,:]
dT_dx_96 = dT_dx[data_1:data_2,:,:]
dT_dy_96 = dT_dy[data_1:data_2,:,:]
Q_net_96 = Q_net[data_1:data_2,:,:]
sst_obser_96 = sst_obser[data_1:data_2,:,:]
analysis_temp_96 = analysis_temp[data_1:data_2,:,:]
eastward_wind_96 = eastward_wind[data_1:data_2,:,:]
northward_wind_96 = northward_wind [data_1:data_2,:,:]
surface_downward_eastward_stress_96 = surface_downward_eastward_stress[data_1:data_2,:,:]
surface_downward_northward_stress_96 = surface_downward_northward_stress[data_1:data_2,:,:]

In [9]:
# MAM   97
date_97_1 = 1520
date_97_2 = 1611
data_1 = date_97_1
data_2 = date_97_2

u_97 = u[data_1:data_2,:,:]
v_97 = v[data_1:data_2,:,:]
v_d_97 = v_d[data_1:data_2,:,:]
T_d_97 = T_d[data_1:data_2,:,:]
u_d_97 = u_d[data_1:data_2,:,:]
mld_97 = mld[data_1:data_2,:,:]
dT_dt_97 = dT_dt[data_1:data_2,:,:]
dT_dx_97 = dT_dx[data_1:data_2,:,:]
dT_dy_97 = dT_dy[data_1:data_2,:,:]
sst_obser_97 = sst_obser[data_1:data_2,:,:]
Q_net_97 = Q_net[data_1:data_2,:,:]
analysis_temp_97 = analysis_temp[data_1:data_2,:,:]
eastward_wind_97 = eastward_wind[data_1:data_2,:,:]
northward_wind_97 = northward_wind [data_1:data_2,:,:]
surface_downward_eastward_stress_97 = surface_downward_eastward_stress[data_1:data_2,:,:]
surface_downward_northward_stress_97 = surface_downward_northward_stress[data_1:data_2,:,:]

In [10]:
# MAM   98
date_98_1 = 1885
date_98_2 = 1977
data_1 = date_98_1
data_2 = date_98_2

u_98 = u[data_1:data_2,:,:]
v_98 = v[data_1:data_2,:,:]
v_d_98 = v_d[data_1:data_2,:,:]
T_d_98 = T_d[data_1:data_2,:,:]
u_d_98 = u_d[data_1:data_2,:,:]
mld_98 = mld[data_1:data_2,:,:]
dT_dt_98 = dT_dt[data_1:data_2,:,:]
dT_dx_98 = dT_dx[data_1:data_2,:,:]
dT_dy_98 = dT_dy[data_1:data_2,:,:]
sst_obser_98 = sst_obser[data_1:data_2,:,:]
Q_net_98 = Q_net[data_1:data_2,:,:]
analysis_temp_98 = analysis_temp[data_1:data_2,:,:]
eastward_wind_98 = eastward_wind[data_1:data_2,:,:]
northward_wind_98 = northward_wind [data_1:data_2,:,:]
surface_downward_eastward_stress_98 = surface_downward_eastward_stress[data_1:data_2,:,:]
surface_downward_northward_stress_98 = surface_downward_northward_stress[data_1:data_2,:,:]

In [11]:
# MAM   99
date_99_1 = 2250
date_99_2 = 2342
data_1 = date_99_1
data_2 = date_99_2

u_99 = u[data_1:data_2,:,:]
v_99 = v[data_1:data_2,:,:]
v_d_99 = v_d[data_1:data_2,:,:]
T_d_99 = T_d[data_1:data_2,:,:]
u_d_99 = u_d[data_1:data_2,:,:]
mld_99 = mld[data_1:data_2,:,:]
dT_dt_99 = dT_dt[data_1:data_2,:,:]
dT_dx_99 = dT_dx[data_1:data_2,:,:]
dT_dy_99 = dT_dy[data_1:data_2,:,:]
sst_obser_99 = sst_obser[data_1:data_2,:,:]
Q_net_99 = Q_net[data_1:data_2,:,:]
analysis_temp_99 = analysis_temp[data_1:data_2,:,:]
eastward_wind_99 = eastward_wind[data_1:data_2,:,:]
northward_wind_99 = northward_wind [data_1:data_2,:,:]
surface_downward_eastward_stress_99 = surface_downward_eastward_stress[data_1:data_2,:,:]
surface_downward_northward_stress_99 = surface_downward_northward_stress[data_1:data_2,:,:]

In [12]:
# MAM   00
date_00_1 = 2616
date_00_2 = 2708
data_1 = date_00_1
data_2 = date_00_2

u_00 = u[data_1:data_2,:,:]
v_00 = v[data_1:data_2,:,:]
v_d_00 = v_d[data_1:data_2,:,:]
T_d_00 = T_d[data_1:data_2,:,:]
u_d_00 = u_d[data_1:data_2,:,:]
mld_00 = mld[data_1:data_2,:,:]
dT_dt_00 = dT_dt[data_1:data_2,:,:]
dT_dx_00 = dT_dx[data_1:data_2,:,:]
dT_dy_00 = dT_dy[data_1:data_2,:,:]
sst_obser_00 = sst_obser[data_1:data_2,:,:]
Q_net_00 = Q_net[data_1:data_2,:,:]
analysis_temp_00 = analysis_temp[data_1:data_2,:,:]
eastward_wind_00 = eastward_wind[data_1:data_2,:,:]
northward_wind_00 = northward_wind [data_1:data_2,:,:]
surface_downward_eastward_stress_00 = surface_downward_eastward_stress[data_1:data_2,:,:]
surface_downward_northward_stress_00 = surface_downward_northward_stress[data_1:data_2,:,:]

In [13]:
# MAM   01
date_01_1 = 2981
date_01_2 = 3073
data_1 = date_01_1
data_2 = date_01_2

u_01 = u[data_1:data_2,:,:]
v_01 = v[data_1:data_2,:,:]
v_d_01 = v_d[data_1:data_2,:,:]
T_d_01 = T_d[data_1:data_2,:,:]
u_d_01 = u_d[data_1:data_2,:,:]
mld_01 = mld[data_1:data_2,:,:]
dT_dt_01 = dT_dt[data_1:data_2,:,:]
dT_dx_01 = dT_dx[data_1:data_2,:,:]
dT_dy_01 = dT_dy[data_1:data_2,:,:]
sst_obser_01 = sst_obser[data_1:data_2,:,:]
Q_net_01 = Q_net[data_1:data_2,:,:]
analysis_temp_01 = analysis_temp[data_1:data_2,:,:]
eastward_wind_01 = eastward_wind[data_1:data_2,:,:]
northward_wind_01 = northward_wind [data_1:data_2,:,:]
surface_downward_eastward_stress_01 = surface_downward_eastward_stress[data_1:data_2,:,:]
surface_downward_northward_stress_01 = surface_downward_northward_stress[data_1:data_2,:,:]

In [14]:
# MAM   02
date_02_1 = 3346
date_02_2 = 3438
data_1 = date_02_1
data_2 = date_02_2

u_02 = u[data_1:data_2,:,:]
v_02 = v[data_1:data_2,:,:]
v_d_02 = v_d[data_1:data_2,:,:]
T_d_02 = T_d[data_1:data_2,:,:]
u_d_02 = u_d[data_1:data_2,:,:]
mld_02 = mld[data_1:data_2,:,:]
dT_dt_02 = dT_dt[data_1:data_2,:,:]
dT_dx_02 = dT_dx[data_1:data_2,:,:]
dT_dy_02 = dT_dy[data_1:data_2,:,:]
sst_obser_02 = sst_obser[data_1:data_2,:,:]
Q_net_02 = Q_net[data_1:data_2,:,:]
analysis_temp_02 = analysis_temp[data_1:data_2,:,:]
eastward_wind_02 = eastward_wind[data_1:data_2,:,:]
northward_wind_02 = northward_wind [data_1:data_2,:,:]
surface_downward_eastward_stress_02 = surface_downward_eastward_stress[data_1:data_2,:,:]
surface_downward_northward_stress_02 = surface_downward_northward_stress[data_1:data_2,:,:]

In [15]:
# MAM   03
date_03_1 = 3711
date_03_2 = 3803
data_1 = date_03_1
data_2 = date_03_2

u_03 = u[data_1:data_2,:,:]
v_03 = v[data_1:data_2,:,:]
v_d_03 = v_d[data_1:data_2,:,:]
T_d_03 = T_d[data_1:data_2,:,:]
u_d_03 = u_d[data_1:data_2,:,:]
mld_03 = mld[data_1:data_2,:,:]
dT_dt_03 = dT_dt[data_1:data_2,:,:]
dT_dx_03 = dT_dx[data_1:data_2,:,:]
dT_dy_03 = dT_dy[data_1:data_2,:,:]
sst_obser_03 = sst_obser[data_1:data_2,:,:]
Q_net_03 = Q_net[data_1:data_2,:,:]
analysis_temp_03 = analysis_temp[data_1:data_2,:,:]
eastward_wind_03 = eastward_wind[data_1:data_2,:,:]
northward_wind_03 = northward_wind [data_1:data_2,:,:]
surface_downward_eastward_stress_03 = surface_downward_eastward_stress[data_1:data_2,:,:]
surface_downward_northward_stress_03 = surface_downward_northward_stress[data_1:data_2,:,:]

In [16]:
# MAM   04
date_04_1 = 4077
date_04_2 = 4169
data_1 = date_04_1
data_2 = date_04_2

u_04 = u[data_1:data_2,:,:]
v_04 = v[data_1:data_2,:,:]
v_d_04 = v_d[data_1:data_2,:,:]
T_d_04 = T_d[data_1:data_2,:,:]
u_d_04 = u_d[data_1:data_2,:,:]
mld_04 = mld[data_1:data_2,:,:]
dT_dt_04 = dT_dt[data_1:data_2,:,:]
dT_dx_04 = dT_dx[data_1:data_2,:,:]
dT_dy_04 = dT_dy[data_1:data_2,:,:]
sst_obser_04 = sst_obser[data_1:data_2,:,:]
Q_net_04 = Q_net[data_1:data_2,:,:]
analysis_temp_04 = analysis_temp[data_1:data_2,:,:]
eastward_wind_04 = eastward_wind[data_1:data_2,:,:]
northward_wind_04 = northward_wind [data_1:data_2,:,:]
surface_downward_eastward_stress_04 = surface_downward_eastward_stress[data_1:data_2,:,:]
surface_downward_northward_stress_04 = surface_downward_northward_stress[data_1:data_2,:,:]

In [17]:
# MAM   05
date_05_1 = 4442
date_05_2 = 4534
data_1 = date_05_1
data_2 = date_05_2

u_05 = u[data_1:data_2,:,:]
v_05 = v[data_1:data_2,:,:]
v_d_05 = v_d[data_1:data_2,:,:]
T_d_05 = T_d[data_1:data_2,:,:]
u_d_05 = u_d[data_1:data_2,:,:]
mld_05 = mld[data_1:data_2,:,:]
dT_dt_05 = dT_dt[data_1:data_2,:,:]
dT_dx_05 = dT_dx[data_1:data_2,:,:]
dT_dy_05 = dT_dy[data_1:data_2,:,:]
sst_obser_05 = sst_obser[data_1:data_2,:,:]
Q_net_05 = Q_net[data_1:data_2,:,:]
analysis_temp_05 = analysis_temp[data_1:data_2,:,:]
eastward_wind_05 = eastward_wind[data_1:data_2,:,:]
northward_wind_05 = northward_wind [data_1:data_2,:,:]
surface_downward_eastward_stress_05 = surface_downward_eastward_stress[data_1:data_2,:,:]
surface_downward_northward_stress_05 = surface_downward_northward_stress[data_1:data_2,:,:]

In [18]:
# MAM   06
date_06_1 = 4807
date_06_2 = 4899
data_1 = date_06_1
data_2 = date_06_2

u_06 = u[data_1:data_2,:,:]
v_06 = v[data_1:data_2,:,:]
v_d_06 = v_d[data_1:data_2,:,:]
T_d_06 = T_d[data_1:data_2,:,:]
u_d_06 = u_d[data_1:data_2,:,:]
mld_06 = mld[data_1:data_2,:,:]
dT_dt_06 = dT_dt[data_1:data_2,:,:]
dT_dx_06 = dT_dx[data_1:data_2,:,:]
dT_dy_06 = dT_dy[data_1:data_2,:,:]
sst_obser_06 = sst_obser[data_1:data_2,:,:]
Q_net_06 = Q_net[data_1:data_2,:,:]
analysis_temp_06 = analysis_temp[data_1:data_2,:,:]
eastward_wind_06 = eastward_wind[data_1:data_2,:,:]
northward_wind_06 = northward_wind [data_1:data_2,:,:]
surface_downward_eastward_stress_06 = surface_downward_eastward_stress[data_1:data_2,:,:]
surface_downward_northward_stress_06 = surface_downward_northward_stress[data_1:data_2,:,:]

In [19]:
# MAM   07
date_07_1 = 5172
date_07_2 = 5264
data_1 = date_07_1
data_2 = date_07_2

u_07 = u[data_1:data_2,:,:]
v_07 = v[data_1:data_2,:,:]
v_d_07 = v_d[data_1:data_2,:,:]
T_d_07 = T_d[data_1:data_2,:,:]
u_d_07 = u_d[data_1:data_2,:,:]
mld_07 = mld[data_1:data_2,:,:]
dT_dt_07 = dT_dt[data_1:data_2,:,:]
dT_dx_07 = dT_dx[data_1:data_2,:,:]
dT_dy_07 = dT_dy[data_1:data_2,:,:]
sst_obser_07 = sst_obser[data_1:data_2,:,:]
Q_net_07 = Q_net[data_1:data_2,:,:]
analysis_temp_07 = analysis_temp[data_1:data_2,:,:]
eastward_wind_07 = eastward_wind[data_1:data_2,:,:]
northward_wind_07 = northward_wind [data_1:data_2,:,:]
surface_downward_eastward_stress_07 = surface_downward_eastward_stress[data_1:data_2,:,:]
surface_downward_northward_stress_07 = surface_downward_northward_stress[data_1:data_2,:,:]

In [20]:
# MAM   08
date_08_1 = 5538
date_08_2 = 5630
data_1 = date_08_1
data_2 = date_08_2

u_08 = u[data_1:data_2,:,:]
v_08 = v[data_1:data_2,:,:]
v_d_08 = v_d[data_1:data_2,:,:]
T_d_08 = T_d[data_1:data_2,:,:]
u_d_08 = u_d[data_1:data_2,:,:]
mld_08 = mld[data_1:data_2,:,:]
dT_dt_08 = dT_dt[data_1:data_2,:,:]
dT_dx_08 = dT_dx[data_1:data_2,:,:]
dT_dy_08 = dT_dy[data_1:data_2,:,:]
sst_obser_08 = sst_obser[data_1:data_2,:,:]
Q_net_08 = Q_net[data_1:data_2,:,:]
analysis_temp_08 = analysis_temp[data_1:data_2,:,:]
eastward_wind_08 = eastward_wind[data_1:data_2,:,:]
northward_wind_08 = northward_wind [data_1:data_2,:,:]
surface_downward_eastward_stress_08 = surface_downward_eastward_stress[data_1:data_2,:,:]
surface_downward_northward_stress_08 = surface_downward_northward_stress[data_1:data_2,:,:]

In [21]:
# MAM   09
date_09_1 = 5903
date_09_2 = 5995
data_1 = date_09_1
data_2 = date_09_2

u_09 = u[data_1:data_2,:,:]
v_09 = v[data_1:data_2,:,:]
v_d_09 = v_d[data_1:data_2,:,:]
T_d_09 = T_d[data_1:data_2,:,:]
u_d_09 = u_d[data_1:data_2,:,:]
mld_09 = mld[data_1:data_2,:,:]
dT_dt_09 = dT_dt[data_1:data_2,:,:]
dT_dx_09 = dT_dx[data_1:data_2,:,:]
dT_dy_09 = dT_dy[data_1:data_2,:,:]
sst_obser_09 = sst_obser[data_1:data_2,:,:]
Q_net_09 = Q_net[data_1:data_2,:,:]
analysis_temp_09 = analysis_temp[data_1:data_2,:,:]
eastward_wind_09 = eastward_wind[data_1:data_2,:,:]
northward_wind_09 = northward_wind [data_1:data_2,:,:]
surface_downward_eastward_stress_09 = surface_downward_eastward_stress[data_1:data_2,:,:]
surface_downward_northward_stress_09 = surface_downward_northward_stress[data_1:data_2,:,:]

In [22]:
# MAM   10
date_10_1 = 6268
date_10_2 = 6360
data_1 = date_10_1
data_2 = date_10_2

u_10 = u[data_1:data_2,:,:]
v_10 = v[data_1:data_2,:,:]
v_d_10 = v_d[data_1:data_2,:,:]
T_d_10 = T_d[data_1:data_2,:,:]
u_d_10 = u_d[data_1:data_2,:,:]
mld_10 = mld[data_1:data_2,:,:]
dT_dt_10 = dT_dt[data_1:data_2,:,:]
dT_dx_10 = dT_dx[data_1:data_2,:,:]
dT_dy_10 = dT_dy[data_1:data_2,:,:]
sst_obser_10 = sst_obser[data_1:data_2,:,:]
Q_net_10 = Q_net[data_1:data_2,:,:]
analysis_temp_10 = analysis_temp[data_1:data_2,:,:]
eastward_wind_10 = eastward_wind[data_1:data_2,:,:]
northward_wind_10 = northward_wind [data_1:data_2,:,:]
surface_downward_eastward_stress_10 = surface_downward_eastward_stress[data_1:data_2,:,:]
surface_downward_northward_stress_10 = surface_downward_northward_stress[data_1:data_2,:,:]

In [23]:
# MAM   11
date_11_1 = 6633
date_11_2 = 6725
data_1 = date_11_1
data_2 = date_11_2

u_11 = u[data_1:data_2,:,:]
v_11 = v[data_1:data_2,:,:]
v_d_11 = v_d[data_1:data_2,:,:]
T_d_11 = T_d[data_1:data_2,:,:]
u_d_11 = u_d[data_1:data_2,:,:]
mld_11 = mld[data_1:data_2,:,:]
dT_dt_11 = dT_dt[data_1:data_2,:,:]
dT_dx_11 = dT_dx[data_1:data_2,:,:]
dT_dy_11 = dT_dy[data_1:data_2,:,:]
sst_obser_11 = sst_obser[data_1:data_2,:,:]
Q_net_11 = Q_net[data_1:data_2,:,:]
analysis_temp_11 = analysis_temp[data_1:data_2,:,:]
eastward_wind_11 = eastward_wind[data_1:data_2,:,:]
northward_wind_11 = northward_wind [data_1:data_2,:,:]
surface_downward_eastward_stress_11 = surface_downward_eastward_stress[data_1:data_2,:,:]
surface_downward_northward_stress_11 = surface_downward_northward_stress[data_1:data_2,:,:]

In [24]:
# MAM   12
date_12_1 = 6999
date_12_2 = 7091
data_1 = date_12_1
data_2 = date_12_2

u_12 = u[data_1:data_2,:,:]
v_12 = v[data_1:data_2,:,:]
v_d_12 = v_d[data_1:data_2,:,:]
T_d_12 = T_d[data_1:data_2,:,:]
u_d_12 = u_d[data_1:data_2,:,:]
mld_12 = mld[data_1:data_2,:,:]
dT_dt_12 = dT_dt[data_1:data_2,:,:]
dT_dx_12 = dT_dx[data_1:data_2,:,:]
dT_dy_12 = dT_dy[data_1:data_2,:,:]
sst_obser_12 = sst_obser[data_1:data_2,:,:]
Q_net_12 = Q_net[data_1:data_2,:,:]
analysis_temp_12 = analysis_temp[data_1:data_2,:,:]
eastward_wind_12 = eastward_wind[data_1:data_2,:,:]
northward_wind_12 = northward_wind [data_1:data_2,:,:]
surface_downward_eastward_stress_12 = surface_downward_eastward_stress[data_1:data_2,:,:]
surface_downward_northward_stress_12 = surface_downward_northward_stress[data_1:data_2,:,:]

In [25]:
# MAM   13
date_13_1 = 7364
date_13_2 = 7455
data_1 = date_13_1
data_2 = date_13_2

u_13 = u[data_1:data_2,:,:]
v_13 = v[data_1:data_2,:,:]
v_d_13 = v_d[data_1:data_2,:,:]
T_d_13 = T_d[data_1:data_2,:,:]
u_d_13 = u_d[data_1:data_2,:,:]
mld_13 = mld[data_1:data_2,:,:]
dT_dt_13 = dT_dt[data_1:data_2,:,:]
dT_dx_13 = dT_dx[data_1:data_2,:,:]
dT_dy_13 = dT_dy[data_1:data_2,:,:]
sst_obser_13 = sst_obser[data_1:data_2,:,:]
Q_net_13 = Q_net[data_1:data_2,:,:]
analysis_temp_13 = analysis_temp[data_1:data_2,:,:]
eastward_wind_13 = eastward_wind[data_1:data_2,:,:]
northward_wind_13 = northward_wind [data_1:data_2,:,:]
surface_downward_eastward_stress_13 = surface_downward_eastward_stress[data_1:data_2,:,:]
surface_downward_northward_stress_13 = surface_downward_northward_stress[data_1:data_2,:,:]

In [26]:
# MAM   14
date_14_1 = 7729
date_14_2 = 7821
data_1 = date_14_1
data_2 = date_14_2

u_14 = u[data_1:data_2,:,:]
v_14 = v[data_1:data_2,:,:]
v_d_14 = v_d[data_1:data_2,:,:]
T_d_14 = T_d[data_1:data_2,:,:]
u_d_14 = u_d[data_1:data_2,:,:]
mld_14 = mld[data_1:data_2,:,:]
dT_dt_14 = dT_dt[data_1:data_2,:,:]
dT_dx_14 = dT_dx[data_1:data_2,:,:]
dT_dy_14 = dT_dy[data_1:data_2,:,:]
sst_obser_14 = sst_obser[data_1:data_2,:,:]
Q_net_14 = Q_net[data_1:data_2,:,:]
analysis_temp_14 = analysis_temp[data_1:data_2,:,:]
eastward_wind_14 = eastward_wind[data_1:data_2,:,:]
northward_wind_14 = northward_wind [data_1:data_2,:,:]
surface_downward_eastward_stress_14 = surface_downward_eastward_stress[data_1:data_2,:,:]
surface_downward_northward_stress_14 = surface_downward_northward_stress[data_1:data_2,:,:]

In [27]:
# MAM   15
date_15_1 = 8094
date_15_2 = 8185
data_1 = date_15_1
data_2 = date_15_2

u_15 = u[data_1:data_2,:,:]
v_15 = v[data_1:data_2,:,:]
v_d_15 = v_d[data_1:data_2,:,:]
T_d_15 = T_d[data_1:data_2,:,:]
u_d_15 = u_d[data_1:data_2,:,:]
mld_15 = mld[data_1:data_2,:,:]
dT_dt_15 = dT_dt[data_1:data_2,:,:]
dT_dx_15 = dT_dx[data_1:data_2,:,:]
dT_dy_15 = dT_dy[data_1:data_2,:,:]
sst_obser_15 = sst_obser[data_1:data_2,:,:]
Q_net_15 = Q_net[data_1:data_2,:,:]
analysis_temp_15 = analysis_temp[data_1:data_2,:,:]
eastward_wind_15 = eastward_wind[data_1:data_2,:,:]
northward_wind_15 = northward_wind [data_1:data_2,:,:]
surface_downward_eastward_stress_15 = surface_downward_eastward_stress[data_1:data_2,:,:]
surface_downward_northward_stress_15 = surface_downward_northward_stress[data_1:data_2,:,:]

In [28]:
# SON   16
date_16_1 = 8460
date_16_2 = 8552
data_1 = date_16_1
data_2 = date_16_2

u_16 = u[data_1:data_2,:,:]
v_16 = v[data_1:data_2,:,:]
v_d_16 = v_d[data_1:data_2,:,:]
T_d_16 = T_d[data_1:data_2,:,:]
u_d_16 = u_d[data_1:data_2,:,:]
mld_16 = mld[data_1:data_2,:,:]
dT_dt_16 = dT_dt[data_1:data_2,:,:]
dT_dx_16 = dT_dx[data_1:data_2,:,:]
dT_dy_16 = dT_dy[data_1:data_2,:,:]
sst_obser_16 = sst_obser[data_1:data_2,:,:]
Q_net_16 = Q_net[data_1:data_2,:,:]
analysis_temp_16 = analysis_temp[data_1:data_2,:,:]
eastward_wind_16 = eastward_wind[data_1:data_2,:,:]
northward_wind_16 = northward_wind [data_1:data_2,:,:]
surface_downward_eastward_stress_16 = surface_downward_eastward_stress[data_1:data_2,:,:]
surface_downward_northward_stress_16 = surface_downward_northward_stress[data_1:data_2,:,:]

In [29]:
# MAM   17
date_17_1 = 8825
date_17_2 = 8917
data_1 = date_17_1
data_2 = date_17_2

u_17 = u[data_1:data_2,:,:]
v_17 = v[data_1:data_2,:,:]
v_d_17 = v_d[data_1:data_2,:,:]
T_d_17 = T_d[data_1:data_2,:,:]
u_d_17 = u_d[data_1:data_2,:,:]
mld_17 = mld[data_1:data_2,:,:]
dT_dt_17 = dT_dt[data_1:data_2,:,:]
dT_dx_17 = dT_dx[data_1:data_2,:,:]
dT_dy_17 = dT_dy[data_1:data_2,:,:]
sst_obser_17 = sst_obser[data_1:data_2,:,:]
Q_net_17 = Q_net[data_1:data_2,:,:]
analysis_temp_17 = analysis_temp[data_1:data_2,:,:]
eastward_wind_17 = eastward_wind[data_1:data_2,:,:]
northward_wind_17 = northward_wind [data_1:data_2,:,:]
surface_downward_eastward_stress_17 = surface_downward_eastward_stress[data_1:data_2,:,:]
surface_downward_northward_stress_17 = surface_downward_northward_stress[data_1:data_2,:,:]

In [30]:
# MAM   18
date_18_1 = 9190
date_18_2 = 9280
data_1 = date_18_1
data_2 = date_18_2

u_18 = u[data_1:data_2,:,:]
v_18 = v[data_1:data_2,:,:]
v_d_18 = v_d[data_1:data_2,:,:]
T_d_18 = T_d[data_1:data_2,:,:]
u_d_18 = u_d[data_1:data_2,:,:]
mld_18 = mld[data_1:data_2,:,:]
dT_dt_18 = dT_dt[data_1:data_2,:,:]
dT_dx_18 = dT_dx[data_1:data_2,:,:]
dT_dy_18 = dT_dy[data_1:data_2,:,:]
sst_obser_18 = sst_obser[data_1:data_2,:,:]
Q_net_18 = Q_net[data_1:data_2,:,:]
analysis_temp_18 = analysis_temp[data_1:data_2,:,:]
eastward_wind_18 = eastward_wind[data_1:data_2,:,:]
northward_wind_18 = northward_wind [9190:9280,:,:]
surface_downward_eastward_stress_18 = surface_downward_eastward_stress[9190:9280,:,:]
surface_downward_northward_stress_18 = surface_downward_northward_stress[9190:9280,:,:]

In [31]:
# SON   19
date_19_1 = 9542
date_19_2 = 9633
data_1 = date_19_1
data_2 = date_19_2

u_19 = u[data_1:data_2,:,:]
v_19 = v[data_1:data_2,:,:]
v_d_19 = v_d[data_1:data_2,:,:]
T_d_19 = T_d[data_1:data_2,:,:]
u_d_19 = u_d[data_1:data_2,:,:]
mld_19 = mld[data_1:data_2,:,:]
dT_dt_19 = dT_dt[data_1:data_2,:,:]
dT_dx_19 = dT_dx[data_1:data_2,:,:]
dT_dy_19 = dT_dy[data_1:data_2,:,:]
sst_obser_19 = sst_obser[data_1:data_2,:,:]
Q_net_19 = Q_net[data_1:data_2,:,:]
analysis_temp_19 = analysis_temp[data_1:data_2,:,:]
eastward_wind_19 = eastward_wind[9542:9633,:,:]
northward_wind_19 = northward_wind [9542:9633,:,:]
surface_downward_eastward_stress_19 = surface_downward_eastward_stress[9542:9633,:,:]
surface_downward_northward_stress_19 = surface_downward_northward_stress[9542:9633,:,:]

In [32]:
u_93_19 = np.concatenate((u_93, u_94, u_95, u_96, u_97, u_98, u_99, u_00, u_01, u_02,
                         u_03, u_04, u_05, u_06, u_07, u_08, u_09, u_10, u_11, u_12, u_13, u_14, u_15,
                         u_16, u_17, u_18, u_19),axis = 0)

In [33]:
v_93_19 = np.concatenate((v_93, v_94, v_95, v_96, v_97, v_98, v_99, v_00, v_01, v_02, v_03,
                         v_04, v_05, v_06, v_07, v_08, v_09, v_10, v_11, v_12, v_13, v_14, v_15, v_16,
                         v_17, v_18, v_19), axis = 0)

In [34]:
v_d_93_19 = np.concatenate((v_d_93, v_d_94, v_d_95, v_d_96, v_d_97, v_d_98,  v_d_99, v_d_00,
                           v_d_01, v_d_02, v_d_03, v_d_04, v_d_05, v_d_06, v_d_07, v_d_08, v_d_09,
                           v_d_10, v_d_11, v_d_12, v_d_13, v_d_14, v_d_15, v_d_16, v_d_17, v_d_18, v_d_19), axis = 0)

In [35]:
T_d_93_19 = np.concatenate((T_d_93, T_d_94, T_d_95, T_d_96, T_d_97, T_d_98, T_d_99, T_d_00,
                           T_d_01, T_d_02, T_d_03, T_d_04, T_d_05, T_d_06, T_d_07, T_d_08, T_d_09, T_d_10,
                           T_d_11, T_d_12, T_d_13, T_d_14, T_d_15, T_d_16, T_d_17, T_d_18, T_d_19), axis = 0)

In [36]:
u_d_93_19 = np.concatenate((u_d_93, u_d_94, u_d_95, u_d_96, u_d_97, u_d_98, u_d_99, u_d_00,
                           u_d_01, u_d_02, u_d_03, u_d_04, u_d_05, u_d_06, u_d_07, u_d_08, u_d_09, u_d_10,
                           u_d_11, u_d_12, u_d_13, u_d_14, u_d_15, u_d_16, u_d_17, u_d_18, u_d_19), axis = 0)

In [37]:
mld_93_19 = np.concatenate((mld_93, mld_94, mld_95, mld_96, mld_97, mld_98, mld_99, mld_00,
                           mld_01, mld_02, mld_03, mld_04, mld_05, mld_06, mld_07, mld_08, mld_09, mld_10,
                           mld_11, mld_12, mld_13, mld_14, mld_15, mld_16, mld_17, mld_18, mld_19), axis = 0)

In [38]:
dT_dt_93_19 = np.concatenate((dT_dt_93, dT_dt_94, dT_dt_95, dT_dt_96, dT_dt_97, dT_dt_98,
                             dT_dt_99, dT_dt_00, dT_dt_01, dT_dt_02, dT_dt_03, dT_dt_04, dT_dt_05, dT_dt_06,
                             dT_dt_07, dT_dt_08, dT_dt_09, dT_dt_10, dT_dt_11, dT_dt_12, dT_dt_13, dT_dt_14,
                             dT_dt_15, dT_dt_16, dT_dt_17, dT_dt_18, dT_dt_19), axis = 0)

In [39]:
dT_dx_93_19 = np.concatenate((dT_dx_93, dT_dx_94, dT_dx_95, dT_dx_96, dT_dx_97, dT_dx_98,
                             dT_dx_99, dT_dx_00, dT_dx_01, dT_dx_02, dT_dx_03, dT_dx_04, dT_dx_05, dT_dx_06,
                             dT_dx_07, dT_dx_08, dT_dx_09, dT_dx_10, dT_dx_11, dT_dx_12, dT_dx_13, dT_dx_14,
                             dT_dx_15, dT_dx_16, dT_dx_17, dT_dx_18, dT_dx_19), axis = 0)

In [40]:
dT_dy_93_19 = np.concatenate((dT_dy_93, dT_dy_94, dT_dy_95, dT_dy_96, dT_dy_97, dT_dy_98,
                             dT_dy_99, dT_dy_00, dT_dy_01, dT_dy_02, dT_dy_03, dT_dy_04, dT_dy_05, dT_dy_06,
                             dT_dy_07, dT_dy_08, dT_dy_09, dT_dy_10, dT_dy_11, dT_dy_12, dT_dy_13, dT_dy_14,
                             dT_dy_15, dT_dy_16, dT_dy_17, dT_dy_18, dT_dy_19), axis = 0)

In [41]:
sst_obser_93_19 = np.concatenate((sst_obser_93, sst_obser_94, sst_obser_95, sst_obser_96, sst_obser_97,
                                 sst_obser_98, sst_obser_99, sst_obser_00, sst_obser_01, sst_obser_02, sst_obser_03,
                                 sst_obser_04, sst_obser_05, sst_obser_06, sst_obser_07, sst_obser_08, sst_obser_09,
                                 sst_obser_10, sst_obser_11, sst_obser_12, sst_obser_13, sst_obser_14, sst_obser_15,
                                 sst_obser_16, sst_obser_17, sst_obser_18, sst_obser_19), axis = 0)

In [42]:
Q_net_93_19 = np.concatenate((Q_net_93, Q_net_94, Q_net_95, Q_net_96, Q_net_97, Q_net_98,Q_net_99,
                             Q_net_00, Q_net_01, Q_net_02, Q_net_03, Q_net_04, Q_net_05, Q_net_06, Q_net_07,
                             Q_net_08, Q_net_09, Q_net_10, Q_net_11, Q_net_12, Q_net_13, Q_net_14, Q_net_15,
                             Q_net_16, Q_net_17, Q_net_18, Q_net_19), axis = 0)

In [43]:
analysis_temp_93_19 = np.concatenate((analysis_temp_93, analysis_temp_94, analysis_temp_95, analysis_temp_96,
                                     analysis_temp_97, analysis_temp_98, analysis_temp_99, analysis_temp_00, analysis_temp_01,
                                     analysis_temp_02, analysis_temp_03, analysis_temp_04, analysis_temp_05, analysis_temp_06,
                                     analysis_temp_07, analysis_temp_08, analysis_temp_09, analysis_temp_10, analysis_temp_11,
                                     analysis_temp_12, analysis_temp_13, analysis_temp_14, analysis_temp_15, analysis_temp_16,
                                     analysis_temp_17, analysis_temp_18, analysis_temp_19), axis = 0)

In [44]:
eastward_wind_93_19 = np.concatenate((eastward_wind_93, eastward_wind_94, eastward_wind_95,
                                     eastward_wind_96, eastward_wind_97, eastward_wind_98, eastward_wind_99,
                                     eastward_wind_00, eastward_wind_01, eastward_wind_02, eastward_wind_03,
                                     eastward_wind_04, eastward_wind_05, eastward_wind_06, eastward_wind_07,
                                     eastward_wind_08, eastward_wind_09, eastward_wind_10, eastward_wind_11,
                                     eastward_wind_12, eastward_wind_13, eastward_wind_14, eastward_wind_15,
                                     eastward_wind_16, eastward_wind_17, eastward_wind_18, eastward_wind_19), axis = 0)

In [45]:
northward_wind_93_19 = np.concatenate((northward_wind_93, northward_wind_94, northward_wind_95,
                                      northward_wind_96, northward_wind_97, northward_wind_98, northward_wind_99,
                                      northward_wind_00, northward_wind_01, northward_wind_02, northward_wind_03,
                                      northward_wind_04, northward_wind_05, northward_wind_06, northward_wind_07,
                                      northward_wind_08, northward_wind_09, northward_wind_10, northward_wind_11,
                                      northward_wind_12, northward_wind_13, northward_wind_14, northward_wind_15,
                                      northward_wind_16, northward_wind_17, northward_wind_18, northward_wind_19), axis = 0)

In [46]:
surface_downward_eastward_stress_93_19 = np.concatenate((surface_downward_eastward_stress_93, surface_downward_eastward_stress_94, surface_downward_eastward_stress_95,
                                                        surface_downward_eastward_stress_96, surface_downward_eastward_stress_97, surface_downward_eastward_stress_98, surface_downward_eastward_stress_99,
                                                        surface_downward_eastward_stress_00, surface_downward_eastward_stress_01, surface_downward_eastward_stress_02, surface_downward_eastward_stress_03,
                                                        surface_downward_eastward_stress_04, surface_downward_eastward_stress_05, surface_downward_eastward_stress_06, surface_downward_eastward_stress_07,
                                                        surface_downward_eastward_stress_08, surface_downward_eastward_stress_09, surface_downward_eastward_stress_10, surface_downward_eastward_stress_11,
                                                        surface_downward_eastward_stress_12, surface_downward_eastward_stress_13, surface_downward_eastward_stress_14, surface_downward_eastward_stress_15,
                                                        surface_downward_eastward_stress_16, surface_downward_eastward_stress_17, surface_downward_eastward_stress_18, surface_downward_eastward_stress_19), axis = 0)

In [47]:
surface_downward_northward_stress_93_19 = np.concatenate((surface_downward_northward_stress_93, surface_downward_northward_stress_94, surface_downward_northward_stress_95, surface_downward_northward_stress_96,
                                                         surface_downward_northward_stress_97, surface_downward_northward_stress_98, surface_downward_northward_stress_99, surface_downward_northward_stress_00, surface_downward_northward_stress_01,
                                                         surface_downward_northward_stress_02, surface_downward_northward_stress_03, surface_downward_northward_stress_04, surface_downward_northward_stress_05,surface_downward_northward_stress_06,
                                                         surface_downward_northward_stress_07, surface_downward_northward_stress_08, surface_downward_northward_stress_09, surface_downward_northward_stress_10, surface_downward_northward_stress_11,
                                                         surface_downward_northward_stress_12, surface_downward_northward_stress_13, surface_downward_northward_stress_14, surface_downward_northward_stress_15,surface_downward_northward_stress_16,
                                                         surface_downward_northward_stress_17, surface_downward_northward_stress_18, surface_downward_northward_stress_19), axis = 0)

In [48]:
np.savez(r'D:\heat_wave\pacific\last\MLD_train\MAM_all_variables.npz',
        u = u_93_19, v = v_93_19, v_d= v_d_93_19, T_d = T_d_93_19, u_d = u_d_93_19, mld = mld_93_19,
        dT_dt = dT_dt_93_19, dT_dx = dT_dx_93_19, dT_dy = dT_dy_93_19, sst_obser = sst_obser_93_19,
        Q_net = Q_net_93_19, analysis_temp = analysis_temp_93_19, eastward_wind = eastward_wind_93_19,
        northward_wind = northward_wind_93_19, 
        surface_downward_eastward_stress = surface_downward_eastward_stress_93_19,
        surface_downward_northward_stress = surface_downward_northward_stress_93_19)